CS4001/4042 Assignment 1, Part B, Q1
---

Real world datasets often have a mix of numeric and categorical features – this dataset is one example. To build models on such data, categorical features have to be encoded or embedded.

PyTorch Tabular is a library that makes it very convenient to build neural networks for tabular data. It is built on top of PyTorch Lightning, which abstracts away boilerplate model training code and makes it easy to integrate other tools, e.g. TensorBoard for experiment tracking.

For questions B1 and B2, the following features should be used:   
- **Numeric / Continuous** features: dist_to_nearest_stn, dist_to_dhoby, degree_centrality, eigenvector_centrality, remaining_lease_years, floor_area_sqm
- **Categorical** features: month, town, flat_model_type, storey_range



---



In [1]:
%pip install pytorch_tabular[extra]

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Wei Kang\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
SEED = 42

import random
random.seed(SEED)
import math
import warnings

import numpy as np
np.random.seed(SEED)

import pandas as pd
from sklearn.metrics import r2_score

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

> Divide the dataset (‘hdb_price_prediction.csv’) into train, validation and test sets by using entries from year 2019 and before as training data, year 2020 as validation data and year 2021 as test data.
**Do not** use data from year 2022 and year 2023.



In [3]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

# Filtering data based on the year
train_df = df[df['year'] <= 2019]
validation_df = df[df['year'] == 2020]
test_df = df[df['year'] == 2021]

# Sanity Check: Get unique values 
train_unique_years = ', '.join(map(str, train_df['year'].unique()))
validation_unique_years = ', '.join(map(str, validation_df['year'].unique()))
test_unique_years = ', '.join(map(str, test_df['year'].unique()))

# Sanity Check: Print the formatted unique values
print(f"Unique years in train_df: {train_unique_years}")
print(f"Unique years in validation_df: {validation_unique_years}")
print(f"Unique years in test_df: {test_unique_years}")

Unique years in train_df: 2017, 2018, 2019
Unique years in validation_df: 2020
Unique years in test_df: 2021


> Refer to the documentation of **PyTorch Tabular** and perform the following tasks: https://pytorch-tabular.readthedocs.io/en/latest/#usage
- Use **[DataConfig](https://pytorch-tabular.readthedocs.io/en/latest/data/)** to define the target variable, as well as the names of the continuous and categorical variables.
- Use **[TrainerConfig](https://pytorch-tabular.readthedocs.io/en/latest/training/)** to automatically tune the learning rate. Set batch_size to be 1024 and set max_epoch as 50.
- Use **[CategoryEmbeddingModelConfig](https://pytorch-tabular.readthedocs.io/en/latest/models/#category-embedding-model)** to create a feedforward neural network with 1 hidden layer containing 50 neurons.
- Use **[OptimizerConfig](https://pytorch-tabular.readthedocs.io/en/latest/optimizer/)** to choose Adam optimiser. There is no need to set the learning rate (since it will be tuned automatically) nor scheduler.
- Use **[TabularModel](https://pytorch-tabular.readthedocs.io/en/latest/tabular_model/)** to initialise the model and put all the configs together.

In [4]:
# TODO: Enter your code here

# 1. Define the DataConfig
data_config = DataConfig(
    target=['resale_price'], 
    continuous_cols=['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm'],
    categorical_cols=['month', 'town', 'flat_model_type', 'storey_range']
)

# 2. Define the TrainerConfig
trainer_config = TrainerConfig(
    auto_lr_find=True, 
    batch_size=1024, 
    max_epochs=50, 
)

# 3. Define the CategoryEmbeddingModelConfig
model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50"  # One hidden layer with 50 neurons.
)

# 4. Create the OptimizerConfig
optimizer_config = OptimizerConfig(optimizer="Adam")

# 5. Create the TabularModel
model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# 6. Train the model
warnings.filterwarnings("ignore")
model.fit(train=train_df, validation=validation_df, seed = SEED)

# 7. Evaluate the model on the test dataset and store the results
result = model.evaluate(test_df)

# 8. Print the evaluation results
print(result)

# 9. Calculate the Root Mean Square Error (RMSE) from the test results
rmse = math.sqrt(result[0]['test_mean_squared_error'])

# 10. Print the RMSE value
print(f"RMSE: {round(rmse,4)}")

# 11. Get predictions from the machine learning model for the test dataset
pred_df = model.predict(test_df)

# 12. Extract the actual and predicted values from the dataframes
actual_values = test_df['resale_price'].values  
predicted_values = pred_df["resale_price_prediction"].values  

# 13. Calculate the coefficient of determination (R²) between actual and predicted values
r2 = r2_score(actual_values, predicted_values)

# 14. Print the R² value, a measure of the model's goodness of fit
print(f"R²: {round(r2,4)}")


2023-10-09 17:20:05,154 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off


Global seed set to 42
2023-10-09 17:20:05,217 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-09 17:20:05,225 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-09 17:20:05,422 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-09 17:20:05,517 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-10-09 17:20:05,570 - {pytorch_tabular.tabular_model:573} - INFO - Auto LR Find Started


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at c:\Users\Wei Kang\Desktop\Individual Assignment\.lr_find_125b9528-71d3-49ec-9a5b-3a7a87d208b7.ckpt
Restored all states from the checkpoint file at c:\Users\Wei Kang\Desktop\Individual Assignment\.lr_find_125b9528-71d3-49ec-9a5b-3a7a87d208b7.ckpt
2023-10-09 17:20:11,813 - {pytorch_tabular.tabular_model:575} - INFO - Suggested LR: 0.5754399373371567. For plot and detailed analysis, use `find_learning_rate` method.
2023-10-09 17:20:11,814 - {pytorch_tabular.tabular_model:582} - INFO - Training Started


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

...

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       5882418176.0        │
│  test_mean_squared_error  │       5882418176.0        │
└───────────────────────────┴───────────────────────────┘

Output()

[{'test_loss': 5882418176.0, 'test_mean_squared_error': 5882418176.0}]
RMSE: 76696.9242


R²: 0.7776


> Report the test RMSE error and the test R2 value that you obtained.



# Test Results

| Metric         | Value (4 d.p.) |
|----------------|---------------:|
| Test RMSE Error| 76696.9242     |
| Test R² Value  | 0.7776         |

> Print out the corresponding rows in the dataframe for the top 25 test samples with the largest errors. Identify a trend in these poor predictions and suggest a way to reduce these errors.



In [6]:
# TODO: Enter your code here

# Step 1: Compute residuals by subtracting predicted resale prices from actual resale prices
pred_df['residuals'] = pred_df['resale_price_prediction'] - test_df['resale_price']

# Step 2: Sort the dataframe by the absolute value of residuals in descending order
pred_df['abs_residuals'] = pred_df['residuals'].abs()
sorted_df = pred_df.sort_values(by='abs_residuals', ascending=False)

# Step 3: Drop unnecessary columns to focus on the main columns
sorted_df.drop(columns=['residuals', 'abs_residuals', 'resale_price_prediction', 'full_address', 'nearest_stn', 'year'], inplace=True)

# Step 4: Get the top 25 samples with the largest errors
top_25_largest_errors = sorted_df.head(25)

# Step 5: Display the results for the top 25 largest errors
display(top_25_largest_errors)

# Step 6: Further exploration of data for the top 25 samples with largest errors:
# Compute mean and median for 'dist_to_dhoby', 'floor_area_sqm', 'remaining_lease_years', count of '3 ROOM' flats, and 'resale_price'
print("For Top 25 Largest Errors:", end='\n\n')
print("Mean 'dist_to_dhoby':", top_25_largest_errors['dist_to_dhoby'].mean())
print("Median 'dist_to_dhoby':", top_25_largest_errors['dist_to_dhoby'].median())

print("Mean 'floor_area_sqm':", top_25_largest_errors['floor_area_sqm'].mean())
print("Median 'floor_area_sqm':", top_25_largest_errors['floor_area_sqm'].median())

print("Mean 'remaining_lease_years':", top_25_largest_errors['remaining_lease_years'].mean())
print("Median 'remaining_lease_years':", top_25_largest_errors['remaining_lease_years'].median())

print("Number of '3 ROOM' flats:", top_25_largest_errors['flat_model_type'].str.contains('3 ROOM').sum())

print("Mean 'resale_price':", top_25_largest_errors['resale_price'].mean())
print("Median 'resale_price':", top_25_largest_errors['resale_price'].median())

# Step 7: Get the top 25 samples with the smallest errors
top_25_smallest_errors = sorted_df.tail(25)

# Step 8: Display the results for the top 25 smallest errors
display(top_25_smallest_errors)

# Step 9: Further exploration of data for the top 25 samples with smallest errors:
# Compute mean and median for 'dist_to_dhoby', 'floor_area_sqm', 'remaining_lease_years', count of '3 ROOM' flats, and 'resale_price'
print("For Top 25 Smallest Errors:", end='\n\n')
print("Mean 'dist_to_dhoby':", top_25_smallest_errors['dist_to_dhoby'].mean())
print("Median 'dist_to_dhoby':", top_25_smallest_errors['dist_to_dhoby'].median())

print("Mean 'floor_area_sqm':", top_25_smallest_errors['floor_area_sqm'].mean())
print("Median 'floor_area_sqm':", top_25_smallest_errors['floor_area_sqm'].median())

print("Mean 'remaining_lease_years':", top_25_smallest_errors['remaining_lease_years'].mean())
print("Median 'remaining_lease_years':", top_25_smallest_errors['remaining_lease_years'].median())

print("Number of '3 ROOM' flats:", top_25_smallest_errors['flat_model_type'].str.contains('3 ROOM').sum())

print("Mean 'resale_price':", top_25_smallest_errors['resale_price'].mean())
print("Median 'resale_price':", top_25_smallest_errors['resale_price'].median())


,month,town,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
92405,11,BUKIT MERAH,0.581977,2.309477,0.016807,0.047782,"3 ROOM, Standard",50.166667,88.0,01 TO 03,780000.0
90957,6,BUKIT BATOK,1.292540,10.763777,0.016807,0.000217,"EXECUTIVE, Apartment",75.583333,144.0,10 TO 12,968000.0
112128,12,TAMPINES,0.370873,12.479752,0.033613,0.000229,"EXECUTIVE, Maisonette",61.750000,148.0,01 TO 03,998000.0
90608,12,BISHAN,0.776182,6.297489,0.033613,0.015854,"5 ROOM, DBSS",88.833333,120.0,37 TO 39,1360000.0
106192,12,QUEENSTOWN,0.658035,3.807573,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.333333,109.0,04 TO 06,968000.0
91871,6,BUKIT MERAH,0.693391,2.058774,0.016807,0.047782,"3 ROOM, Standard",50.583333,88.0,01 TO 03,680888.0
93825,8,CENTRAL AREA,0.451637,2.594828,0.016807,0.103876,"5 ROOM, Adjoined flat",54.583333,118.0,16 TO 18,938000.0
92504,12,BUKIT MERAH,0.468378,2.365532,0.016807,0.047782,"3 ROOM, Standard",50.166667,88.0,01 TO 03,695000.0
105695,6,QUEENSTOWN,0.745596,3.720593,0.016807,0.008342,"4 ROOM, Premium Apartment Loft",93.916667,97.0,07 TO 09,930000.0
90432,8,BISHAN,0.827889,6.370404,0.033613,0.015854,"5 ROOM, DBSS",88.916667,120.0,25 TO 27,1280000.0


For Top 25 Largest Errors:

Mean 'dist_to_dhoby': 4.39348055137666
Median 'dist_to_dhoby': 3.720593002068167
Mean 'floor_area_sqm': 109.96
Median 'floor_area_sqm': 113.0
Mean 'remaining_lease_years': 74.73333333333333
Median 'remaining_lease_years': 78.08333333333333
Number of '3 ROOM' flats: 6
Mean 'resale_price': 966183.04
Median 'resale_price': 968000.0


,month,town,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
98512,3,JURONG EAST,1.318285,12.291372,0.025210,0.000550,"4 ROOM, Model A",94.666667,92.0,22 TO 24,525000.0
88682,2,BEDOK,0.562867,7.523270,0.016807,0.001743,"EXECUTIVE, Apartment",71.833333,143.0,01 TO 03,713000.0
92233,10,BUKIT MERAH,0.782082,3.170640,0.016807,0.047782,"3 ROOM, Standard",47.333333,63.0,04 TO 06,268000.0
109494,1,SERANGOON,1.667455,8.644621,0.016807,0.003825,"5 ROOM, Improved",75.583333,121.0,01 TO 03,490000.0
108114,7,SENGKANG,0.447635,11.958252,0.016807,0.000233,"5 ROOM, Improved",78.000000,116.0,13 TO 15,490000.0
91749,5,BUKIT MERAH,1.002651,3.845544,0.016807,0.047665,"4 ROOM, Improved",54.000000,82.0,07 TO 09,400000.0
88973,5,BEDOK,0.386846,8.410227,0.016807,0.001743,"3 ROOM, Simplified",64.833333,64.0,01 TO 03,305000.0
101243,12,KALLANG/WHAMPOA,0.757741,2.647546,0.016807,0.053004,"3 ROOM, Improved",57.666667,66.0,10 TO 12,307000.0
106718,1,SENGKANG,1.333543,12.125693,0.016807,0.000233,"3 ROOM, Model A",95.416667,68.0,04 TO 06,330000.0
112376,4,TOA PAYOH,1.099275,4.446539,0.016807,0.017995,"3 ROOM, Improved",48.416667,66.0,04 TO 06,240000.0


For Top 25 Smallest Errors:

Mean 'dist_to_dhoby': 7.792203830376139
Median 'dist_to_dhoby': 7.523270252583895
Mean 'floor_area_sqm': 84.04
Median 'floor_area_sqm': 68.0
Mean 'remaining_lease_years': 64.98
Median 'remaining_lease_years': 59.0
Number of '3 ROOM' flats: 15
Mean 'resale_price': 395147.52
Median 'resale_price': 330000.0


## Observed Trend

### Model Underperformance for Premium Houses in Central Locations

The primary trend we've discerned is that our model ***struggles to accurately predict the resale prices of houses that are more expensive and located in central areas***. This observation is supported by the following data insights:

- **Distance to Dhoby Ghaut (dist_to_dhoby)**
  - Mean for Largest Errors: `4.39 km`
  - Mean for Smallest Errors: `7.79 km` <br/><br/>
  > First, it's observed that the distance to Dhoby Ghaut Station (dist_to_dhoby) tends to be smaller for the records with larger prediction errors. Specifically, the mean distance for these samples is approximately 4.39 km, compared to 7.79 km for those with smaller errors. This suggests that the model is less adept at predicting resale prices for flats closer to Dhoby Ghaut. 

- **Floor Area (floor_area_sqm)**
  - Mean for Largest Errors: `109.96 sqm`
  - Mean for Smallest Errors: `84.04 sqm` <br/><br/>
  > Secondly, the floor area (floor_area_sqm) of flats also demonstrates a pattern: those with larger errors have a higher mean floor area of 109.96 sqm, as opposed to 84.04 sqm for those with smaller errors. This discrepancy suggests that the model may not be effectively capturing the impact of floor area on resale price. 

- **Flat Type ('3 ROOM')**
  - Largest Errors: `6 out of 25`
  - Smallest Errors: `15 out of 25` <br/><br/>
  > Thirdly, a higher concentration of '3 ROOM' flats appears in the bottom 25 rows with smaller errors, constituting 15 out of 25 samples. In contrast, only 6 out of the top 25 rows with the largest errors are '3 ROOM' flats.

- **Remaining Lease Years (remaining_lease_years)**
  - Mean for Largest Errors: `74.73 years`
  - Mean for Smallest Errors: `64.98 years` <br/><br/>
  > Lastly, the remaining lease years of flats also demonstrates a pattern: those with larger errors have a higher mean remaining lease year of 74.73 years, compared to 64.98 years for those with smaller errors. This discrepancy suggests that the model may not be effectively capturing the impact of remaining lease years on resale price.

- **Resale Price**
  - Mean for Resale Price for Largest Errors: `$966,183.04`
  - Mean for Resale Price for Smallest Errors: `$395,147.52` <br/><br/>
  > The resale price in the smallest errors cases is also markedly lower, indicating that the model performs better at predicting lower-valued properties.

## Recommendations for Model Improvement

1. **Feature Transformation for Distance to Central Location**
  - Firstly, we can consider applying logarithmic or polynomial transformations on `dist_to_dhoby` to capture its nonlinear impact on resale prices.

2. **Incorporate Interaction Terms**
  - Secondly, we can introduce interaction terms for `floor_area_sqm` with features like town or flat type, capturing compounded effects of size and location/type on pricing.

3. **Enhance Model Complexity and Architecture**
  - Current Architecture: 
    - Single hidden layer with 50 neurons
    - Trainer Config: max_epochs=50, batch_size=1024
    - Optimizer: Adam
  - Thirdly, in the context of the PyTorch Tabular model, adjusting the trainer configurations such as max_epochs and batch_size could also be considered. A smaller batch_size might provide a more accurate estimation of the gradient, whereas increasing max_epochs may provide the model more time to converge to a better solution. Additionally, experimenting with different optimizers like RMSprop or SGD could provide different optimization characteristics that may be more suitable for the data distribution at hand. These trainer and optimizer configurations should be carefully tuned and validated to see which combination yields the most accurate predictions.

4. **Regularization**:
  - Fourthly, we can choose to incorporate either L1 or L2 regularization to further prevent overfitting on top of techniques like Dropout Layers and Early Stopping to help the model generalize better to unseen data of resale flats.

5. **Data Augmentation**
  - Lastly, for better accuracy with premium properties, we can also consider data augmentation techniques, such as synthetic data generation for expensive houses, to refine predictions.
